---

### for me

 - [YAML Cheatsheet](https://quickref.me/yaml.html)
 - [YAML Viewer](https://jsonformatter.org/yaml-viewer)

# Notes

 - provide links and URIs wherever possible
 - _extended_ YAML with inline links in Markdown style [text](https://www.example.org)
 - 

# Feedback

 - the more you annotate _what_ the information in the guides _is_ (types, meta-info), the more information we can extract and potentially link together with the Datahub
 - try to keep structures _across_ guides of different levels _and_ pieces of information as uniform as possible
   -> makes info _predictable_ for others and allows to link more into the Datahub
 - 

# Problems

 - which URIs?
   -> definitely the Datahub ones _at some point_ but those aren't ready yet
  - which thesaurus/thesauri?

# Thoughts, Ideas

 - link directly to other search guides (across levels)
 - link directly to example objects that the current guide pertains to (is examplary of)
   -> this serves the "points of acces"/"portals" motivation of the research guides

---

# YAML to Markdown Parsing - custom-built

In [1]:
from glob import glob
import yaml

from ResearchAids import ResearchAid

BASE_DIR = "../published"
eng = glob(f"{BASE_DIR}/*/English/*.yml")
dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
# top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = sorted(dutch + eng)

for filename in yaml_files:
    with open(filename) as handle:
        yml = yaml.safe_load(handle)
        try:
            r = ResearchAid(yml, raise_parsing_error=True)
        except KeyError as e:
            print(filename, e)
            # raise
            if "remarks" in str(e):
                print(filename, e)
                raise
            else:
                print(filename, e)
        except AttributeError as e:
            print(filename, e)



../published/niveau3/English/NZG_20240508.yml 'Link'
../published/niveau3/English/NZG_20240508.yml 'Link'


In [ ]:
for y in yamls:
    print(y.keys())

In [ ]:
# print(YAML2MD(yamls[1])())
# yamls[3]["Relevant data"]#["Tags"]
print([s.keys() for s in yamls[2]["Sources"]['Secondary sources']][0], "\n---")
print([s.keys() for s in yamls[3]["Sources"]['Secondary sources']][0])



In [ ]:
# print(Level2(yamls[2])())

print(ResearchAid(yamls[0], raise_parsing_error=True)())

In [ ]:
with open("../published/niveau3/English/NZG_20240508.yml") as handle:
    yml = yaml.safe_load(handle)

[d.keys() for d in yml["Sources"]['Secondary sources']]
yml["Sources"]['Secondary sources']

---

In [34]:
import re
from glob import glob
import yaml


def correct_IRI(url):
    # correct IRIs:
    #  - https://sws.geonames.org/6255149/
    #  - http://vocab.getty.edu/aat/300266789
    # -- http://www.wikidata.org/entity/Q219477
    md_link_re = re.compile(r"\[(.*)\]\(https?:\/\/(?:sws|www).geonames.org\/([0-9]+)\/?.*\)")
    # uri_re = re.compile(r"^https?:\/\/(?:sws|www).geonames.org\/([0-9]+)\/?.*")

    if md_link_re.match(url):
        link_text, geonames_id = md_link_re.match(url).group(1), md_link_re.match(url).group(2)
        print(f"parsed {url}")
        return f"[{link_text}](https://sws.geonames.org/{geonames_id}/)"
    elif ("http" in url[:20]) or ("www" in url[:20]):
        print(f" {url}  didn't parse! is it correct?")
    else:
        pass
    
    return url



complex_types = (list, dict)
def iter_urls(yml):
    if isinstance(yml, str):
        return correct_IRI(yml)
    if isinstance(yml, list):
        return list(map(iter_urls, yml))
    if isinstance(yml, dict):
        return {iter_urls(k): iter_urls(v) for k, v in yml.items()}
    return yml

In [35]:
BASE_DIR = "../published"
eng = glob(f"{BASE_DIR}/*/English/*.yml")
dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
# top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = sorted(dutch + eng)

for filename in yaml_files:
    print(filename)
    with open(filename) as handle:
        yml = yaml.safe_load(handle)
        iter_urls(yml)
    print("\n-------------------\n")

../published/niveau0/Dutch/TopLevel_20240606.yml

-------------------

../published/niveau0/English/TopLevel_20240606.yml

-------------------

../published/niveau1/Dutch/DoingResearch_20240425.yml

-------------------

../published/niveau1/Dutch/Reporting_20240501.yml

-------------------

../published/niveau1/Dutch/RestitutionPolicy_20250123.yml

-------------------

../published/niveau1/Dutch/SelectAndDelineate_20240425.yml

-------------------

../published/niveau1/Dutch/Sources_20240425.yml

-------------------

../published/niveau1/English/DoingResearch_20240425.yml

-------------------

../published/niveau1/English/Reporting_20240501.yml

-------------------

../published/niveau1/English/RestitutionPolicy_20250123.yml

-------------------

../published/niveau1/English/SelectAndDelineate_20240425.yml

-------------------

../published/niveau1/English/Sources_20240501.yml

-------------------

../published/niveau2/Dutch/ArubaBonaireCuracao_20250619.yml
parsed [Aruba](https://sws.g

In [38]:
with open("../published/niveau0/Dutch/TopLevel_20240606.yml") as handle:
    yml = yaml.safe_load(handle)
    print(yml == iter_urls(yml))


True
